In [14]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from rdkit import Chem
from rdkit.Chem import rdDetermineBonds, AllChem
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import pandas as pd
import os
from loguru import logger

from strain_relief import compute_strain

## Running StrainRelief 

In [15]:
# First, lets generate some example poses with 3D coordinates.
smiles = ["C", "CC"]
poses = []

for s in smiles:
    mol = Chem.MolFromSmiles(s)
    mol = Chem.AddHs(mol)
    AllChem.EmbedMolecule(mol)
    poses.append(mol)

# Note: to run StrainRelief your molecules must either have specified 
# bonds or they must be able to be passed through RDKit's rdDetermineBonds 
# function. This is needed for the conformer enumeration.

In [16]:
# We now need to initialise a run configuration.
# We use MMFF94s here so that runs will only take a few seconds.
with initialize(version_base="1.1", config_path="../hydra_config"):
    cfg = compose(
        config_name="default", 
        overrides=["experiment=pytest", "calculator.model_paths=../models/MACE_SPICE2_NEUTRAL.model"]
    )

print(OmegaConf.to_yaml(cfg))

seed: -1
threshold: 16.1
num_workers: 0
device: cpu
batch_size: -1
local_optimiser:
  fmax: 0.75
  _target_: neural_optimiser.optimisers.BFGS
  max_step: 0.04
  steps: 250
  fexit: 250
io:
  input:
    parquet_path: null
    mol_col_name: null
    id_col_name: null
    include_charged: true
  output:
    parquet_path: null
    mol_col_name: ${..input.mol_col_name}
    id_col_name: ${..input.id_col_name}
    molecule_attr: id
conformers:
  randomSeed: ${seed}
  numConfs: 1
  maxAttempts: 10
  pruneRmsThresh: 0.1
  clearConfs: false
  numThreads: ${num_workers}
calculator:
  _target_: neural_optimiser.calculators.MACECalculator
  model_paths: ../models/MACE_SPICE2_NEUTRAL.model
  device: ${device}
  default_dtype: float32
global_optimiser:
  _target_: neural_optimiser.optimisers.BFGS
  max_step: 0.04
  steps: 250
  fmax: 0.5
  fexit: 250



StrainRelief is run either via the `compute_strain` function or via the command line with the `strain-relief` command. The following three examples demonstrate different ways of running the tool, all giving the same output.

In [17]:
# EXAMPLE 1

# The minimal requirement to run StrainRelief is a list of RDKit.Mols with 3D poses and a run configration.
# If ids are not given then they will be generated.
results = compute_strain(mols=poses, ids=None, cfg=cfg)
results.head()

CONFIG
├── seed
│   └── -1                                                                                                         
├── threshold
│   └── 16.1                                                                                                       
├── num_workers
│   └── 0                                                                                                          
├── device
│   └── cpu                                                                                                        
├── batch_size
│   └── -1                                                                                                         
├── local_optimiser
│   └── fmax: 0.75                                                                                                 
│       _target_: neural_optimiser.optimisers.BFGS                                                                 
│       max_step: 0.04                                                                                             
│       steps: 250                                                                                                 
│       fexit: 250                                                                                                 
│                                                                                                                  
├── io
│   └── input:                                                                                                     
│         parquet_path: null                                                                                       
│         mol_col_name: null                                                                                       
│         id_col_name: null                                                                                        
│         include_charged: true                                                                                    
│       output:                                                                                                    
│         parquet_path: null                                                                                       
│         mol_col_name: ${..input.mol_col_name}                                                                    
│         id_col_name: ${..input.id_col_name}                                                                      
│         molecule_attr: id                                                                                        
│                                                                                                                  
├── conformers
│   └── randomSeed: ${seed}                                                                                        
│       numConfs: 1                                                                                                
│       maxAttempts: 10                                                                                            
│       pruneRmsThresh: 0.1                                                                                        
│       clearConfs: false                                                                                          
│       numThreads: ${num_workers}                                                                                 
│                                                                                                                  
├── calculator
│   └── _target_: neural_optimiser.calculators.MACECalculator                                                      
│       model_paths: ../models/MACE_SPICE2_NEUTRAL.model                                                           
│       device: ${device}                                                                                          
│       default_dtype: float32                                                                                     
│                                          

2025-10-26 18:22:17.183 | WARNING  | strain_relief.configs._configs:_validate_paths:27 - No output path provided, results will not be saved to disk
2025-10-26 18:22:17.183 | WARNING  | strain_relief.configs._configs:_validate_calculator:79 - MACE (v0.3.14) currently has limited support for charged molecules.
2025-10-26 18:22:17.184 | INFO     | strain_relief.compute_strain:compute_strain:73 - Instantiating calculator...
2025-10-26 18:22:17.254 | INFO     | strain_relief.compute_strain:compute_strain:75 - MACECalculator(model_paths=../models/MACE_SPICE2_NEUTRAL.model, device=cpu, max_neighbours=32, default_dtype='torch.float32')
2025-10-26 18:22:17.258 | INFO     | strain_relief.compute_strain:compute_strain:84 - Instantiating local optimiser...
2025-10-26 18:22:17.259 | DEBUG    | neural_optimiser.optimisers.base:__init__:75 - Initialized BFGS(max_step=0.04, steps=250, fmax=0.75, fexit=250)
2025-10-26 18:22:17.259 | INFO     | strain_relief.compute_strain:compute_strain:86 - BFGS(max_s

,id,mol_bytes,formal_charge,spin_multiplicity,local_min_mol,local_min_e,global_min_mol,global_min_e,ligand_strain,passes_strain_filter
0,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,0,1,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-25436.978021,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-25436.978021,0.000000,True
1,1,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,0,1,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-50117.495751,"b""\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...",-50117.884222,0.388471,True


In [18]:
# EXAMPLE 2

# Instead of a list of mols, StrainRelief can be passed a dataframe with a "mol_bytes" and "id" column.
# "mol_bytes" is a column of RDKit.Mol objects stroed as bytes.
# Any additional columns, such as "smiles" here are also returned.

df = pd.DataFrame([{"mol_bytes": mol.ToBinary(), **mol.GetPropsAsDict()} for mol in poses])
df = df.reset_index(drop=False, names='id')

results2 = compute_strain(df=df, cfg=cfg)
results2.head()

CONFIG
├── seed
│   └── -1                                                                                                         
├── threshold
│   └── 16.1                                                                                                       
├── num_workers
│   └── 0                                                                                                          
├── device
│   └── cpu                                                                                                        
├── batch_size
│   └── 1000000000                                                                                                 
├── local_optimiser
│   └── fmax: 0.75                                                                                                 
│       _target_: neural_optimiser.optimisers.BFGS                                                                 
│       max_step: 0.04                                                                                             
│       steps: 250                                                                                                 
│       fexit: 250                                                                                                 
│                                                                                                                  
├── io
│   └── input:                                                                                                     
│         parquet_path: null                                                                                       
│         mol_col_name: null                                                                                       
│         id_col_name: null                                                                                        
│         include_charged: true                                                                                    
│       output:                                                                                                    
│         parquet_path: null                                                                                       
│         mol_col_name: ${..input.mol_col_name}                                                                    
│         id_col_name: ${..input.id_col_name}                                                                      
│         molecule_attr: id                                                                                        
│                                                                                                                  
├── conformers
│   └── randomSeed: ${seed}                                                                                        
│       numConfs: 1                                                                                                
│       maxAttempts: 10                                                                                            
│       pruneRmsThresh: 0.1                                                                                        
│       clearConfs: false                                                                                          
│       numThreads: ${num_workers}                                                                                 
│                                                                                                                  
├── calculator
│   └── _target_: neural_optimiser.calculators.MACECalculator                                                      
│       model_paths: ../models/MACE_SPICE2_NEUTRAL.model                                                           
│       device: ${device}                                                                                          
│       default_dtype: float32                                                                                     
│                                          

2025-10-26 18:22:17.526 | WARNING  | strain_relief.configs._configs:_validate_paths:27 - No output path provided, results will not be saved to disk
2025-10-26 18:22:17.527 | WARNING  | strain_relief.configs._configs:_validate_calculator:79 - MACE (v0.3.14) currently has limited support for charged molecules.
2025-10-26 18:22:17.527 | INFO     | strain_relief.compute_strain:compute_strain:73 - Instantiating calculator...
2025-10-26 18:22:17.585 | INFO     | strain_relief.compute_strain:compute_strain:75 - MACECalculator(model_paths=../models/MACE_SPICE2_NEUTRAL.model, device=cpu, max_neighbours=32, default_dtype='torch.float32')
2025-10-26 18:22:17.586 | INFO     | strain_relief.compute_strain:compute_strain:84 - Instantiating local optimiser...
2025-10-26 18:22:17.587 | DEBUG    | neural_optimiser.optimisers.base:__init__:75 - Initialized BFGS(max_step=0.04, steps=250, fmax=0.75, fexit=250)
2025-10-26 18:22:17.587 | INFO     | strain_relief.compute_strain:compute_strain:86 - BFGS(max_s

,id,mol_bytes,formal_charge,spin_multiplicity,local_min_mol,local_min_e,global_min_mol,global_min_e,ligand_strain,passes_strain_filter
0,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,0,1,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-25436.978021,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-25436.978021,0.000000,True
1,1,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,0,1,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-50117.495751,"b""\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...",-50117.884222,0.388471,True


In [22]:
# EXAMPLE 3

# Finally, StrainRelief can be run from the command line by specifying an input parquet path
# There are more example scripts in StrainRelief/examples/
df.to_parquet("../data/tutorial_example.parquet")
! strain-relief io.input.parquet_path=../data/tutorial_example.parquet io.output.parquet_path=../data/tutorial_output.parquet experiment=mace calculator.model_paths=../models/MACE_SPICE2_NEUTRAL.model

if os.path.exists("../data/tutorial_output.parquet"):
    results3 = pd.read_parquet("../data/tutorial_output.parquet")
else:
    logger.warning("Output parquet not found, command line example may have failed.")

results3.head()

2025-10-26 18:23:37.621 | INFO     | strain_relief.io._input:load_parquet:46 - Loading data...
2025-10-26 18:23:37.639 | INFO     | strain_relief.io._input:load_parquet:48 - Loaded 2 posed molecules
2025-10-26 18:23:37.640 | INFO     | strain_relief.io._input:_check_columns:128 - RDKit.Mol column is 'mol'
2025-10-26 18:23:37.640 | INFO     | strain_relief.io._input:_check_columns:134 - ID column is 'id'
2025-10-26 18:23:37.641 | INFO     | strain_relief.io._input:_calculate_charge:155 - Dataset contains 0 charged molecules.
CONFIG
├── seed
│   └── -1                                                                      
├── threshold
│   └── 16.1                                                                    
├── num_workers
│   └── 0                                                                       
├── device
│   └── cpu                                                                     
├── batch_size
│   └── -1                                                                

,id,mol_bytes,formal_charge,spin_multiplicity,local_min_mol,local_min_e,global_min_mol,global_min_e,ligand_strain,passes_strain_filter
0,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,0,1,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-25436.978021,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-25437.096251,0.118230,True
1,1,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,0,1,"b""\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...",-50117.884222,"b""\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...",-50120.941321,3.057099,True


## Examining the Output

In [24]:
! strain-relief io.input.parquet_path=../data/example_ligboundconf_input.parquet io.output.parquet_path=../data/example_ligboundconf_output.parquet experiment=mace calculator.model_paths=../models/MACE_SPICE2_NEUTRAL.model
lig = pd.read_parquet("../data/example_ligboundconf_output.parquet")
lig.head()

2025-10-26 18:24:14.135 | INFO     | strain_relief.io._input:load_parquet:46 - Loading data...
2025-10-26 18:24:14.151 | INFO     | strain_relief.io._input:load_parquet:48 - Loaded 2 posed molecules
2025-10-26 18:24:14.151 | INFO     | strain_relief.io._input:_check_columns:128 - RDKit.Mol column is 'mol'
2025-10-26 18:24:14.151 | INFO     | strain_relief.io._input:_check_columns:134 - ID column is 'id'
2025-10-26 18:24:14.152 | INFO     | strain_relief.io._input:_calculate_charge:155 - Dataset contains 0 charged molecules.
CONFIG
├── seed
│   └── -1                                                                      
├── threshold
│   └── 16.1                                                                    
├── num_workers
│   └── 0                                                                       
├── device
│   └── cpu                                                                     
├── batch_size
│   └── -1                                                                

,id,mol_bytes,ligand_id,some_property,formal_charge,spin_multiplicity,local_min_mol,local_min_e,global_min_mol,global_min_e,ligand_strain,passes_strain_filter
0,0,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,3Q4_3QD0_A_370,A,0,1,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-906454.923152,"b""\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...",-906460.147807,5.224655,True
1,1,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,YTW_6IBK_A_525,B,0,1,b'\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...,-787330.347660,"b""\xef\xbe\xad\xde\x00\x00\x00\x00\x10\x00\x00...",-787352.057004,21.709344,False


The `lig` dataframe contains all input columns (in this case `id`, `mol_bytes` and `some_property`) and all calculated columns:
- `formal_charge` (int): RDKit's formal charge
- `local_min_mol` (bytes): the coordinates of the local minimum
- `local_min_e` (float): the energy of the local minimum (in kcal/mol)
- `global_min_mol` (bytes): the coordinates of the global minimum
- `global_min_e` (float): the energy of the global minimum (in kcal/mol)
- `ligand_strain` (float): difference between local and global minima
- `passes_strain_filter` (bool): whether `ligand_strain` is lower than the config threshold
- `nconfs_converged` (int): the number of conformers that convereged when searching for the global minimum 

Lets have a look at the three poses from ligand 3Q4_3QD0_A_370. 

In [9]:
docked = Chem.Mol(lig.mol_bytes[0])
local_min = Chem.Mol(lig.local_min_mol[0])
global_min = Chem.Mol(lig.global_min_mol[0])

In [10]:
rdDetermineBonds.DetermineBonds(docked)
rdDetermineBonds.DetermineBonds(local_min)
rdDetermineBonds.DetermineBonds(global_min)

In [11]:
IPythonConsole.drawMol3D(docked)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [12]:
IPythonConsole.drawMol3D(local_min)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [13]:
IPythonConsole.drawMol3D(global_min)

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

The original and local minimum conformers look very similar to the eye. This is because local minimisation has a loose convergence criteria and is simply to clean up any high energy artifacts left by docking. The global minimum is noticably different, with all aromatic rings having relaxed into a similar plane.

Hopefully you now have a good grasp on how to run the StrainRelief tool! I hope you find it as useful as we have.

Ewan